In [1]:
import os
import cv2
import math
import mediapipe as mp
import csv

import torch
from torch import nn
mp_pose = mp.solutions.pose

def findDistance(x1, y1, x2, y2):
    dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return dist

def findAngle(x1, y1, x2, y2):
    if (x2 - x1) == 0 or y1 == 0:
        return 0
    theta = math.acos((y2 - y1) * (-y1) / (math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) * y1))
    degree = int(180 / math.pi) * theta
    return degree


In [2]:

def process_image(image, pose):
    if image is None:
        print(f"Failed to read image")
        return

    h, w = image.shape[:2]
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    keypoints = pose.process(image_rgb)
    lm = keypoints.pose_landmarks

    if lm is not None:
        nose_x = lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
        nose_y = lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
        nose_z = lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

        l_shldr_x = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
        l_shldr_y = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
        l_shldr_z = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
        r_shldr_x = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
        r_shldr_y = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
        r_shldr_z=lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h

        # Write posture data to CSV file
        return [nose_x, nose_y, nose_z, l_shldr_x,l_shldr_y,l_shldr_z,r_shldr_x,r_shldr_y,r_shldr_z]
    else:
        print(f"No landmarks detected in image ")


In [3]:
class slouch_detection(nn.Module):
    def __init__(self):
        super(slouch_detection,self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(9,36),
            nn.ReLU(),
            nn.Linear(36,15),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(15,12),
            nn.ReLU(),
            nn.Linear(12,6),
            nn.ReLU(),
            nn.Linear(6,3),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(3,1),
            nn.Sigmoid()
        )
    def forward(self,x):
        return self.layers(x)

In [4]:
def slouch_detector(x):
    model=slouch_detection()
    model.load_state_dict(torch.load("slouch_detector.pt"))
    ans=model(x)
    return ans.item()

In [5]:

def main():
    # Initialize the webcam
    cap = cv2.VideoCapture(0)  # 0 represents the default webcam, change it if you have multiple webcams
    
    # Check if the webcam is opened correctly
    if not cap.isOpened():
        print("Error: Unable to access the webcam.")
        return
    
    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()
        
        # Check if the frame is captured correctly
        if not ret:
            print("Error: Unable to capture frame.")
            break
        
        pose=mp_pose.Pose()
        x = process_image(frame,pose)
        print(x)
        res= slouch_detector(torch.tensor(x))
        print(res)
        # Display the captured frame
        cv2.imshow('Webcam', frame)
        
        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release the webcam and close all OpenCV windows
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


[310.6682777404785, 168.45903396606445, -801.4001083374023, 531.4417266845703, 391.75357818603516, -242.64141082763672, 89.79331970214844, 392.92150497436523, -295.200891494751]
0.5303101539611816


Qt: Session management error: Could not open network socket


[311.14110946655273, 165.6603240966797, -808.9202499389648, 525.8244705200195, 396.28149032592773, -264.40587043762207, 89.40208435058594, 396.7123603820801, -296.5463447570801]
0.0
[311.605281829834, 168.20932388305664, -809.2672348022461, 522.8911209106445, 394.8541259765625, -239.66392993927002, 87.1194076538086, 395.40919303894043, -311.1654281616211]
0.0
[312.33251571655273, 167.5630760192871, -779.4868469238281, 521.8622970581055, 396.44433975219727, -217.1730136871338, 84.08021926879883, 394.8175048828125, -313.0766201019287]
0.0
[312.69222259521484, 167.68243789672852, -719.2831993103027, 524.3268966674805, 404.70508575439453, -220.0563097000122, 87.50724792480469, 394.9709129333496, -281.661901473999]
0.0
[317.3860549926758, 164.4403839111328, -691.054859161377, 532.28759765625, 399.8083305358887, -214.63271141052246, 86.43091201782227, 398.4346389770508, -275.96766471862793]
0.0
[315.2573585510254, 161.32261276245117, -733.5836219787598, 529.8509979248047, 394.27831649780273,